In [280]:
import math
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim

In [281]:
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")

This model approximates the function f(x) = 1/1+e^-x

In [282]:
import struct

def int_to_64bit_list(number):
    binary_str = format(number, '064b')
    return [int(bit) for bit in binary_str]

def float_to_128bit_list(number):
    binary_str = format(int(number * 2**64), '064b').zfill(128)
    return [int(bit) for bit in binary_str]

In [283]:
# test dataset
logistic = lambda x: 1 / (1 + math.exp(-x))

ints = [x for x in range(10_000)]

input = [int_to_64bit_list(x) for x in ints]

labels = [float_to_128bit_list(logistic(x)) for x in ints]


In [284]:
t = torch.Tensor(input)
t[0]

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [285]:
class Data(Dataset):
    def __init__(self, inputs, outputs):
        self.inputs = inputs
        self.labels = outputs

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return torch.tensor(self.inputs[idx], dtype=torch.float32), torch.tensor(self.labels[idx], dtype=torch.float32)

In [286]:
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()  
        self.layer1 = nn.Linear(64, 512 * 5) # 64 bits in
        self.layer2 = nn.Linear(512 * 5, 512 * 10)  
        self.output = nn.Linear(512 * 10, 128) # 64 bits out

    def forward(self, x):
        # try relu to begin with
        x = torch.relu(self.layer1(x))
        x = torch.relu(self.layer2(x))
        x = self.output(x)
        return x

In [287]:
training_dataset = Data(input, labels)
training_loader = torch.utils.data.DataLoader(training_dataset,batch_size=128)

model = Model().to(device)

loss = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

for epoch in range(10):
    for data in training_loader:
        inputs, labels = data
        optimizer.zero_grad()
        outputs = model(inputs.to(device))
        loss_size = loss(outputs, labels.to(device))
        loss_size.backward()
        optimizer.step()

    print(f"Loss: {loss_size.item()}")

torch.save(model.state_dict(), "model")

Loss: -0.0
Loss: 2.0116564769523393e-07
Loss: 2.2351740014414645e-08
Loss: -0.0
Loss: 1.4901160305669237e-08
Loss: -0.0
Loss: -0.0
Loss: -0.0
Loss: -0.0
Loss: -0.0
